## Лаб № 9: Комбинированный метод штрафных  функций

In [8]:
import numpy as np
import copy
#import matplotlib.pyplot as plt

In [9]:
# Fonction principale
def f(x):
    return (x[0]-5)**2 + (x[1]-1)**2 + (x[2]+2)**2

In [59]:
# Fonction Shtraf (штрафовая функция)
# Nous avons deux conditions:
# 1.) x1 = 4                          <==> x1-4 = 0
# 2.) (x2 + 1)^2 + (x3 + 1)^2 >= 11   <==>  11 - (x2 + 1)^2 - (x3 + 1)^2 <= 0

# Condition No1
def c1(x):
    return x[0]-4
#Condition No2
def c2(x):
    return 11 - (x[1]+1)**2 - (x[2]+1)**2

# Fonction Shtraf    
def P(x,R):
    return ((c1(x)**2)/(2*R)) - R / c2(x)    

In [55]:
# Fonction auxilliaire
def Phi(x,R):
    return f(x) + P(x,R)

## Nous utilisons la methode du Лаб №3: алгоритм Нелдера-Мида

In [56]:
# Cette fonction va chercher le point du minimum x_min


# x0 -> c'est le point de depart
# step -> le pas qu'on fait vers l'avant

# max_iter -> le nombre maximal d'iterations (d'essais)
# no_improve_limit -> si en allant de l'avant f(x) n'augmente pas plus "no_improve_limit" 
#                     alors on considere au'il n'y a pas d'amelioration

# no_improv_stop = 10 -> Si apres 10 essais il n'y a toujours pas d'amelioration, on arrete l'algorithm

# alpha, beta, gamma ce sont des parametres

def nelder_mead(f, x0,
                step=0.1, no_improve_limit=10e-6,
                no_improv_stop=10, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
   
    
    n = len(x0) # Nombre de variables (dans notre cas ce sera n = 3)
    prev_best = f(x0) # La valeur de f au point x0
    no_improv = 0
    res = [[x0, prev_best]] # Ici on va garder le meilleur point avec la valeur de f

    for i in range(n):         # Pour chaque x_i
        x = copy.copy(x0)
        x[i] = x[i] + step     # On avance un peu
        score = f(x)           # On regarde la valeur de f(x)
        res.append([x, score]) # On ajoute a "res"

    # On fait des iterations
    iters = 0
    while 1:
        # On range f1 < f2 < ....< fn+1
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # Si on depassons max_iter
        if max_iter and iters >= max_iter:
            return res[0]
        iters += 1

        if best < prev_best - no_improve_limit:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1
        
        # Si nous depassons 10 tentavives sans amelioration on arrete
        if no_improv >= no_improv_stop:
            return res[0]

        # On prend un point moyen parmi ceux qu'on a deja
        x0 = [0.] * n
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # A PARTIR D'ICI IL Y A TROIS POSSIBILITES
        # 1 - REFLECTION
        # 2 - EXPANSION
        # 3 - CONTRACTION
        
        # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # expansion
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # contraction
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # APRES NOUS REMPLACONS TOUS LES POINTS DANS RES PAR DE NOUVEAU
        # X_i = X1 + sigma * (X_i - X1) 
        
        x1 = res[0][0] # Le meilleur point, la ou f(x) est plus petit
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres

In [60]:
# Methode de combinaison
def combine_method(n,x0,R0=1,beta=4,epsilon=0.05,kmax=10):  # ETAPE 1
    k=0
    Rk = R0
    xk = copy.copy(x0)
    
    while k < kmax:
        pk = lambda x: Phi(x,Rk) # ETAPE 2
        
        x_Rk,_ = nelder_mead(pk,x0=np.array(xk)) # ETAPE 3
        
        # ETAPE 4
        print(f"k={k} xk = {xk}")
        if (np.abs(P(x_Rk,Rk)) < epsilon):
            print("Solution founded!")
            return x_Rk
        else:
            Rk /= beta
            xk = x_Rk
        k += 1
    return xk

In [61]:
combine_method(n=3,x0=[1,10,10])

k=0 xk = [1, 10, 10]
k=1 xk = [ 1. 10. 10.]
k=2 xk = [ 4.33346986  1.01430675 -2.00727306]
k=3 xk = [ 4.11097159  1.00303346 -2.00167967]
Solution founded!


array([ 4.03028519,  1.00076168, -2.0002702 ])